In [ ]:
from openai import OpenAI
import requests
import json
from functions import convert_pdfs_to_pngs, get_the_files_with_paths, encode_image, validate_json
import os
import re
from dotenv import load_dotenv
from datetime import datetime

In [ ]:
# Załaduj zmienne środowiskowe z pliku .env
load_dotenv()

# Przypisz zmienne środowiskowe do zmiennych
api_key = os.getenv('OPENAI_API')
pdf_path = os.getenv('PDF_DIRECTORY')
png_path = os.getenv('OUTPUT_DIRECTORY')
main_json_file = os.getenv('JSON_MAIN_FILE')

In [ ]:
# Konwertowanie PDF'ów na PNG z folderu PDFs do folderu PNGs jak i zmienianie im nazwy na prefix 'Converted' i datę utworzenia pliku.
convert_pdfs_to_pngs(pdf_path, png_path)

# Wybieranie plików do OCR'owania z katalogu PNG, które nie zaczynają się od DONE
files_to_analyze = get_the_files_with_paths(png_path)

print(f"Liczba plików do Sprawdzenia: {len(files_to_analyze)}")

In [ ]:
# Jeden pliczek do testów na razie
file  = files_to_analyze[-2]
file

In [ ]:
# Getting the base64 string for GPT
base64_image = encode_image(file)

OCR and Image Recognition with GPT

In [ ]:
# Otwórz plikt z promptem i odczytaj jego zawartość jako string
with open('prompt.txt', 'r', encoding='utf-8') as file:
    prompt = file.read()

In [ ]:
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": f"{prompt}"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ]
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

In [ ]:
# Get the interesting part
content = response.json()['choices'][0]['message']['content']

# Remove the ```json and ``` markers
cleaned_content = content.strip('```json\n').strip('\n```')

# Parse the cleaned string as JSON
json_data = json.loads(cleaned_content)

# Print the parsed JSON object to verify
print(json.dumps(json_data, indent=4, ensure_ascii=False))

In [ ]:
validate_json(json_data)

In [ ]:
# Wczytaj istniejące transakcje, jeśli plik istnieje
if os.path.exists(main_json_file):
    with open(main_json_file, 'r', encoding='utf-8') as file:
        all_transactions = json.load(file)
else:
    all_transactions = {"transactions": []}

# Dodaj nowe transakcje do zbiorczego pliku
all_transactions["transactions"].extend(json_data["transactions"])

# Zapisz zaktualizowany plik
with open(main_json_file, 'w', encoding='utf-8') as file:
    json.dump(all_transactions, file, ensure_ascii=False, indent=4)

print(f"Zaktualizowany plik zapisano jako {main_json_file}")